In [1]:
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_image
from functools import partial
from rasterio.enums import MergeAlg
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
buildings = gpd.read_file('building_halle.geojson')

In [3]:
buildings.head()

,id,ID_BUILD,ID_SOURCE,HEIGHT_WALL,HEIGHT_ROOF,NB_LEV,TYPE,MAIN_USE,ZINDEX,ID_BLOCK,ID_RSU,geometry
0,78118,78119,w180289203,3.554208,3.554208,1,residential,residential,0,72349,14742,"POLYGON ((11.99822 51.42148, 11.99824 51.42143..."
1,78119,78120,w180289215,6.576059,6.576059,2,church,religious,0,72350,14742,"POLYGON ((11.99848 51.42183, 11.99847 51.42174..."
2,78046,78047,w200420911,6.393401,6.393401,2,residential,residential,0,12448,14742,"POLYGON ((12.00237 51.42180, 12.00224 51.42186..."
3,50970,50971,w180289193,7.057839,7.057839,2,residential,residential,0,57654,14742,"POLYGON ((11.99915 51.42180, 11.99899 51.42177..."
4,78060,78061,w200420925,5.148638,5.148638,1,residential,residential,0,12448,14742,"POLYGON ((12.00225 51.42187, 12.00221 51.42188..."


In [4]:
buildings.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
# Reproject to EPSG:25832 (ETRS89/UTM zone 32N). See https://epsg.io/25832
# rep_buildings = buildings.to_crs('EPSG:25832')
# Reproject to EPSG:3035 - ETRS89-extended / LAEA Europe
rep_buildings = buildings.to_crs('EPSG:3035')

In [6]:
rep_buildings.crs

<Derived Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Turkey; United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Europ

In [7]:
heights = rep_buildings[["HEIGHT_WALL", "geometry"]]

In [8]:
type(heights)

geopandas.geodataframe.GeoDataFrame

In [9]:
heights.head()

,HEIGHT_WALL,geometry
0,3.554208,"POLYGON ((4459976.180 3147533.667, 4459977.527..."
1,6.576059,"POLYGON ((4459992.754 3147573.422, 4459992.543..."
2,6.393401,"POLYGON ((4460263.415 3147577.621, 4460254.391..."
3,7.057839,"POLYGON ((4460039.448 3147570.961, 4460028.925..."
4,5.148638,"POLYGON ((4460254.906 3147584.467, 4460252.325..."


In [10]:
# Rasterize
rastheights = make_geocube(vector_data=heights, measurements=['HEIGHT_WALL'], resolution=[-10, 10], fill=65535)

In [11]:
rastheights.rio.to_raster('geoclimate_heights.tiff')